In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microsoft-lmr-dataset/SampleSubmission.csv
/kaggle/input/microsoft-lmr-dataset/manifest-cc0c941470ece0a5db57d20166f03edb20240516-20567-1looi1p.json
/kaggle/input/microsoft-lmr-dataset/Train.csv
/kaggle/input/microsoft-lmr-dataset/Test.csv


In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.4 MB/s eta 0:00:00


In [3]:
import spacy
from textblob import TextBlob
import contractions
import re
import warnings
warnings.filterwarnings(action="ignore")
import string
from tqdm import tqdm
from nltk.corpus import stopwords
#from wordsegment import load, segment
from collections import Counter
from wordcloud import STOPWORDS, WordCloud
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
#from sklearn.utils.class_weight import compute_class_weight
#from sklearn.model_selection import train_test_split, StratifiedKFold
#import torch
#from torch.utils.data import Dataset
#from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [5]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(device)

In [4]:
data_doc = pd.read_csv('/kaggle/input/microsoft-lmr-dataset/Train.csv')
print(data_doc.shape)

(73072, 3)


In [5]:
data_doc.head()

,tweet_id,text,location
0,ID_1001136212718088192,NaN,EllicottCity
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland


In [6]:
test_doc = pd.read_csv('/kaggle/input/microsoft-lmr-dataset/Test.csv')
print(test_doc.shape, end='\n\n')
test_doc.head(3)

(2942, 2)



,tweet_id,text
0,ID_1001154804658286592,What is happening to the infrastructure in New...
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...


In [7]:
#sub_doc = pd.read_csv('/kaggle/input/microsoft-lmr-dataset/SampleSubmission.csv')
#sub_doc.head(3)

In [7]:
data_doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73072 entries, 0 to 73071
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  73072 non-null  object
 1   text      16448 non-null  object
 2   location  43460 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


There are 56,624 missing entries in the text column and 29,612 missing entries from the location column
Approaches to try:
1. Drop missing entries in the text since we have no information on them
2. Generate missing text with prompt engineering
3. For text entries with missing locations, use NER to extract the location from the text

In [8]:
data_doc = data_doc.dropna(subset=['text'])
data_doc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16448 entries, 1 to 73071
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  16448 non-null  object
 1   text      16448 non-null  object
 2   location  11849 non-null  object
dtypes: object(3)
memory usage: 514.0+ KB


In [9]:
data_doc.head(3)

,tweet_id,text,location
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland


In [10]:
data_doc.dropna(inplace=True)
data_doc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11849 entries, 1 to 73071
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  11849 non-null  object
 1   text      11849 non-null  object
 2   location  11849 non-null  object
dtypes: object(3)
memory usage: 370.3+ KB


In [12]:
# View 10 random entries from the data
sampled_data = data_doc.sample(n=20)

for i, row in sampled_data.iterrows():
    print(f"Entry {i+1}:")
    print(f"Text: {row['text']}")
    print(f"Location: {row['location']}")
    print("-" * 50)



Entry 45847:
Text: BERNAMA - 1.3 Million Children Desperately Need Help After Cyclone Mora  #lka #srilanka
Location: srilanka
--------------------------------------------------
Entry 14124:
Text: While the United States dealt with #Florence, the Hong Kong had severe flooding from #Mangkhut -- causing this car to float!
Location: Hong Kong United States
--------------------------------------------------
Entry 10828:
Text: Horry County officials and emergency staff are providing live updates on #HurricaneFlorence now. Watch LIVE on our Facebook page now.
Location: Horry County
--------------------------------------------------
Entry 36180:
Text: 6.1 magnitude quake rocks parts of Pakistan. Prayers for the earliest recovery of injured. Hope the damage is not colossal. #earthquake
Location: Pakistan
--------------------------------------------------
Entry 65382:
Text: Hurricane Maria: Horrific images of destruction and misery are emerging from Puerto Rico. @AshleeMullany #7News
Location: P

In [ ]:
# Load the spaCy model
#nlp = spacy.load("en_core_web_lg")

#define a function to extract the locations from the corresponding text
#def extract_locations(text):
    #doc = nlp(text)
#    locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
  #  return locations if locations else None

#data_doc['location'] = data_doc.apply(lambda x: extract_locations(x['text']) if pd.isnull(x['location']) else x['location'],
 #                                axis=1)

In [13]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_lg")

# Define a function to extract locations from the text
#def extract_locations(text):
 #   doc = nlp(text)
 #   locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
 #   return locations if locations else ["No location"]

#data_doc['location'] = data_doc.apply(
#    lambda x: extract_locations(x['text']) if pd.isnull(x['location']) else x['location'],
#    axis=1)

# Flatten the list of locations and ensure it's a single string
#data_doc['location'] = data_doc['location'].apply(lambda loc: loc[0] if isinstance(loc, list) else loc)


In [14]:
x = data_doc[data_doc['location']=='RT']
x

,tweet_id,text,location


In [15]:
data_doc.isnull().sum()

tweet_id    0
text        0
location    0
dtype: int64

In [16]:
data_doc['location'].value_counts()

location
Mexico                           582
California                       544
Ecuador                          539
Nebraska                         495
Haiti                            424
                                ... 
Nebraska Columbia County           1
Nebraska Lincoln Fremont           1
Iowa Wisconsin Nebraska            1
Florida Mexico Nebraska Texas      1
Israels Mexico City Thailand       1
Name: count, Length: 3476, dtype: int64

In [19]:
data_doc['text'][73068], data_doc['location'][73068]

('Rescue workers recover the body of the last person known to be missing after the Mexico City earthquake',
 'Mexico City')

In [17]:
class PreprocessText:
    def __init__(self, nlp):
        self.nlp = nlp
        self.nltk_stopwords = set(stopwords.words("english"))
        self.spacy_stopwords = set(self.nlp.Defaults.stop_words)
        self.combined_stopwords = self.nltk_stopwords.union(self.spacy_stopwords)

    def simplify_punctuation_and_whitespace(self, text):
        text = self._replace_urls(text)
        text = self._remove_punctuation(text)
        text = self._normalize_whitespace(text)
        text = self._remove_digits(text)
        text = text.lower()
        text = contractions.fix(text)  # Fix contractions
        return text

    def _replace_urls(self, text):
        url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
        text = re.sub(url_regex, "", text)
        text = re.sub(r'@\w+', '', text)  # Remove @ mentions
        return text

    def _remove_punctuation(self, text):
        return text.translate(str.maketrans('', '', string.punctuation))

    def _normalize_whitespace(self, text):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()

    def _remove_digits(self, text):
        return re.sub(r'\d+', '', text)

    def lemmatize(self, text):
        return self._lemmatize_text(text)

    def _lemmatize_text(self, text):
        doc = self.nlp(text)
        lemmatized_text = " ".join([token.lemma_ for token in doc])
        return lemmatized_text
    
    def remove_emojis(self, text):
        # Define the emoji pattern using regex
        emoji_pattern = re.compile(
            "["       
            "\U0001F600-\U0001F64F"  # Emoticons
            "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
            "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
            "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
            "\U00002700-\U000027BF"  # Dingbats
            "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
            "\U00002600-\U000026FF"  # Miscellaneous Symbols
            "\U00002B50-\U00002B55"  # Stars
            "\U00002300-\U000023FF"  # Miscellaneous Technical
            "\U00002B05-\U00002B07"  # Arrows
            "\U00002B1B-\U00002B1C"  # Squares
            "]+", flags=re.UNICODE
        )
        return emoji_pattern.sub(r'', text)
    
    def remove_hashtags(self, text):
        return re.sub(r'#(\w+)', r'\1', text)

    def normalization_pipeline(self, text):
        text = self.simplify_punctuation_and_whitespace(text)
        text = self.remove_hashtags(text)  # Remove hashtags
        text = self.lemmatize(text)
        text = " ".join(text.split())  # Normalize whitespace
    
        # Remove stopwords
        text = [i for i in text.split() if i not in self.combined_stopwords]
        
        # Remove emojis
        text = self.remove_emojis(" ".join(text))
        
        # Remove non-ASCII characters
        text = [i for i in text.split() if i.isascii()]
        
        return " ".join(text)



In [18]:
text_preprocessor = PreprocessText(nlp)
data_doc["clean_text"] = data_doc["text"].apply(text_preprocessor.normalization_pipeline)
test_doc["clean_text"] = test_doc["text"].apply(text_preprocessor.normalization_pipeline)

In [43]:
# View 20 random entries from the data
cleaned_data = data_doc.sample(n=20)

for i, row in cleaned_data.iterrows():
    print(f"Entry {i+1}:")
    print(f"Clean Text: {row['clean_text']}")
    print(f"Location: {row['location']}")
    print("-" * 130)


Entry 25666:
Clean Text: I raise money flood victim nebraska click donate
Location: Nebraska
----------------------------------------------------------------------------------------------------------------------------------
Entry 42705:
Clean Text: wake america clintons foundation look donation utilize benefit haiti trump pence
Location: HAITI
----------------------------------------------------------------------------------------------------------------------------------
Entry 56194:
Clean Text: update powerful cat irma pack mph wind hurricane warning issue bahama
Location: Bahamas
----------------------------------------------------------------------------------------------------------------------------------
Entry 37831:
Clean Text: rt powerful magnitude earthquake strike ecuador dead village totally devastate
Location: Ecuador
----------------------------------------------------------------------------------------------------------------------------------
Entry 13665:
Clean Text: h

In [42]:
# Correct the specific location error
data_doc['location'] = data_doc['location'].replace('Puerto Ricoalready', 'Puerto Rico')
data_doc['location'] = data_doc['location'].replace('Ecuadors', 'Ecuador')
data_doc['location'] = data_doc['location'].replace('FortMacFire Alberta', 'Alberta')
data_doc['location'] = data_doc['location'].replace('Carolinas', 'Carolina')
data_doc['location'] = data_doc['location'].replace('the Carolinas Virgin Islands Puerto Rico',
                                                    'Carolina Virgin Islands Puerto Rico')
data_doc['location'] = data_doc['location'].replace('NEW ZEALAND New Zealand', 'New Zealand')

In [44]:
clean = data_doc.copy()
location_counts = clean['location'].value_counts()
frequent_locations = location_counts[location_counts > 1].index

# Filter the DataFrame to keep only rows with frequent location
clean_doc = clean[clean['location'].isin(frequent_locations)]

In [45]:
clean_doc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9025 entries, 1 to 73071
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_id    9025 non-null   object
 1   text        9025 non-null   object
 2   location    9025 non-null   object
 3   clean_text  9025 non-null   object
dtypes: object(4)
memory usage: 352.5+ KB


In [46]:
clean_doc.to_csv('/kaggle/working/new_clean_data_doc.csv')
test_doc.to_csv('/kaggle/working/new_clean_test_doc.csv')